GAN: https://vbarra.github.io/DLbook/gan.html
VAE: https://vbarra.github.io/DLbook/vae.html

# 1. Environment Setup

Using Python 3.9.21

In [4]:
!pip install tensorflow numpy matplotlib urllib3 jupyter tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 MB 5.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 7.6 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 9.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.4/128.4 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.8/417.8 kB 7.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 10.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 9.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 8.6 MB

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

print("TensorFlow version:", tf.__version__)
print("NumPy version:", np.__version__)

ModuleNotFoundError: No module named 'tensorflow'

# Data Loading and Preparation

In [2]:
# @title Download Quick, Draw! Data (.npy files)

import os

# Danh sách 5 class bạn muốn sử dụng (sửa lại nếu cần)
classes = ["key", "dog", "bicycle", "tree", "cloud"]

# Đường dẫn đến thư mục bạn muốn lưu dữ liệu
base_data_dir = "quickdraw_data"

# Tạo thư mục nếu chưa tồn tại
os.makedirs(base_data_dir, exist_ok=True)

# Lặp qua từng class và tải file .npy
for class_name in classes:
    print(f"Downloading {class_name}.npy...")
    
    # Tạo đường dẫn đầy đủ đến file .npy
    file_url = f"https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/{class_name}.npy"
    
    # Tạo đường dẫn lưu file
    save_path = os.path.join(base_data_dir, f"{class_name}.npy")
    
    # Sử dụng !wget để tải file
    !wget -O "$save_path" "$file_url"

print("Download completed!")

zsh:1: command not found: wget
zsh:1: command not found: wget
zsh:1: command not found: wget
zsh:1: command not found: wget
zsh:1: command not found: wget
Download completed!
